In [1]:
import os

import pandas as pd
import numpy as np
import sqlalchemy
import pymysql
import json
import csv

from flask import Flask
from flask import jsonify
from flask import request
from flask import make_response
from flask import url_for
from flask import render_template

from sqlalchemy.orm.exc import NoResultFound
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker

In [8]:
engine = create_engine("sqlite:///db/big_mart_db.db")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
# Create our session (link) from Python to the DB
Session = sessionmaker(bind=engine)

# Save references to db table
Train_data = Base.classes.train
Test_data = Base.classes.test

In [14]:
session = Session()
train_results = session.query(Train_data.primary_key, Train_data.item_fat_content,Train_data.item_identifier, Train_data.item_MRP
                             ,Train_data.item_outlet_sales,Train_data.item_type, Train_data.item_weight, Train_data.outlet_identifier
                             ,Train_data.outlet_location_type,Train_data.outlet_size,Train_data.outlet_type,Train_data.source
                             ,Train_data.outlet_years,Train_data.item_visibility_mean_ratio).all()
session.close()

In [15]:
train_df = pd.DataFrame(train_results)
train_df.head()

,primary_key,item_fat_content,item_identifier,item_MRP,item_outlet_sales,item_type,item_weight,outlet_identifier,outlet_location_type,outlet_size,outlet_type,source,outlet_years,item_visibility_mean_ratio
0,0,Low Fat,FDA15,249.8092,3735.1380,Food,9.30,OUT049,Tier 1,Medium,Supermarket Type1,train,14,1.000000
1,1,Regular,DRC01,48.2692,443.4228,Drink,5.92,OUT018,Tier 3,Medium,Supermarket Type2,train,4,1.002515
2,2,Low Fat,FDN15,141.6180,2097.2700,Food,17.50,OUT049,Tier 1,Medium,Supermarket Type1,train,14,0.998508
3,3,Regular,FDX07,182.0950,732.3800,Food,19.20,OUT010,Tier 3,Small,Grocery Store,train,15,0.778300
4,4,Non-Edible,NCD19,53.8614,994.7052,Non-consumable,8.93,OUT013,Tier 3,High,Supermarket Type1,train,26,0.742195


In [16]:
#jsonfy the df
train_json = json.loads(train_df.to_json(orient='records'))

#check
train_json

#add this into python app:
#jsonify(train_json)

[{'item_MRP': 249.8092,
  'item_fat_content': 'Low Fat',
  'item_identifier': 'FDA15',
  'item_outlet_sales': 3735.138,
  'item_type': 'Food',
  'item_visibility_mean_ratio': 1.0,
  'item_weight': 9.3,
  'outlet_identifier': 'OUT049',
  'outlet_location_type': 'Tier 1',
  'outlet_size': 'Medium',
  'outlet_type': 'Supermarket Type1',
  'outlet_years': 14,
  'primary_key': 0,
  'source': 'train'},
 {'item_MRP': 48.2692,
  'item_fat_content': 'Regular',
  'item_identifier': 'DRC01',
  'item_outlet_sales': 443.4228,
  'item_type': 'Drink',
  'item_visibility_mean_ratio': 1.0025149437,
  'item_weight': 5.92,
  'outlet_identifier': 'OUT018',
  'outlet_location_type': 'Tier 3',
  'outlet_size': 'Medium',
  'outlet_type': 'Supermarket Type2',
  'outlet_years': 4,
  'primary_key': 1,
  'source': 'train'},
 {'item_MRP': 141.618,
  'item_fat_content': 'Low Fat',
  'item_identifier': 'FDN15',
  'item_outlet_sales': 2097.27,
  'item_type': 'Food',
  'item_visibility_mean_ratio': 0.9985084995,
  'i

In [19]:
session = Session()
#querying train table from sqlite
test_results = session.query(Test_data.primary_key, Test_data.item_fat_content,Test_data.item_identifier, Test_data.item_MRP
                             ,Test_data.item_outlet_sales,Test_data.item_type, Test_data.item_weight, Test_data.outlet_identifier
                             ,Test_data.outlet_location_type,Test_data.outlet_size,Test_data.outlet_type,Test_data.source
                             ,Test_data.outlet_years,Test_data.item_visibility_mean_ratio).all()
    #close session
session.close()

In [21]:
test_df = pd.DataFrame(test_results)
test_df.head()

,primary_key,item_fat_content,item_identifier,item_MRP,item_outlet_sales,item_type,item_weight,outlet_identifier,outlet_location_type,outlet_size,outlet_type,source,outlet_years,item_visibility_mean_ratio
0,8523,Low Fat,FDW58,107.8622,,Food,20.750,OUT049,Tier 1,Medium,Supermarket Type1,test,14,1.000000
1,8524,Regular,FDW14,87.3198,,Food,8.300,OUT017,Tier 2,Small,Supermarket Type1,test,6,1.005656
2,8525,Non-Edible,NCN55,241.7538,,Non-consumable,14.600,OUT010,Tier 3,Small,Grocery Store,test,15,1.671197
3,8526,Low Fat,FDQ58,155.0340,,Food,7.315,OUT017,Tier 2,Small,Supermarket Type1,test,6,1.005656
4,8527,Regular,FDY38,234.2300,,Food,13.600,OUT027,Tier 3,Medium,Supermarket Type3,test,28,0.993613


In [22]:
test_json = json.loads(test_df.to_json(orient='records'))
test_json

[{'item_MRP': 107.8622,
  'item_fat_content': 'Low Fat',
  'item_identifier': 'FDW58',
  'item_outlet_sales': '',
  'item_type': 'Food',
  'item_visibility_mean_ratio': 1.0,
  'item_weight': 20.75,
  'outlet_identifier': 'OUT049',
  'outlet_location_type': 'Tier 1',
  'outlet_size': 'Medium',
  'outlet_type': 'Supermarket Type1',
  'outlet_years': 14,
  'primary_key': 8523,
  'source': 'test'},
 {'item_MRP': 87.3198,
  'item_fat_content': 'Regular',
  'item_identifier': 'FDW14',
  'item_outlet_sales': '',
  'item_type': 'Food',
  'item_visibility_mean_ratio': 1.0056564069,
  'item_weight': 8.3,
  'outlet_identifier': 'OUT017',
  'outlet_location_type': 'Tier 2',
  'outlet_size': 'Small',
  'outlet_type': 'Supermarket Type1',
  'outlet_years': 6,
  'primary_key': 8524,
  'source': 'test'},
 {'item_MRP': 241.7538,
  'item_fat_content': 'Non-Edible',
  'item_identifier': 'NCN55',
  'item_outlet_sales': '',
  'item_type': 'Non-consumable',
  'item_visibility_mean_ratio': 1.6711967287,
  'i

In [53]:
# Summary data
# We found that there is huge gap between max and 75% of Item_Outlets_Sales which mean there must be some outliers in Sales
train_summary = train_df.describe()
#reset index in order to stats column
train_summary = train_summary.reset_index()
#rename index to stats
train_summary.rename(columns={'index': 'stats'}, inplace=True)
#drop primary_key column
#train_summary.drop('primary_key', axis=1, inplace=True)

train_summary

,stats,item_MRP,item_outlet_sales,item_weight,outlet_years,item_visibility_mean_ratio
0,count,8523.000000,8523.000000,8523.000000,8523.000000,8523.000000
1,mean,140.992782,2181.288914,12.873623,15.168133,1.080369
2,std,62.275067,1706.499616,4.645893,8.371760,0.237589
3,min,31.290000,33.290000,4.555000,4.000000,0.558097
4,25%,93.826500,834.247400,8.785000,9.000000,0.998023
5,50%,143.012800,1794.331000,12.600000,14.000000,1.000000
6,75%,185.643700,3101.296400,16.850000,26.000000,1.003294
7,max,266.888400,13086.964800,21.350000,28.000000,3.010094


In [54]:
train_summary_json = json.loads(train_summary.to_json(orient='records'))
train_summary_json 

[{'item_MRP': 8523.0,
  'item_outlet_sales': 8523.0,
  'item_visibility_mean_ratio': 8523.0,
  'item_weight': 8523.0,
  'outlet_years': 8523.0,
  'stats': 'count'},
 {'item_MRP': 140.9927819782,
  'item_outlet_sales': 2181.288913575,
  'item_visibility_mean_ratio': 1.0803693083,
  'item_weight': 12.8736225507,
  'outlet_years': 15.1681332864,
  'stats': 'mean'},
 {'item_MRP': 62.2750665122,
  'item_outlet_sales': 1706.4996157338,
  'item_visibility_mean_ratio': 0.2375892603,
  'item_weight': 4.6458926291,
  'outlet_years': 8.3717604081,
  'stats': 'std'},
 {'item_MRP': 31.29,
  'item_outlet_sales': 33.29,
  'item_visibility_mean_ratio': 0.5580971738,
  'item_weight': 4.555,
  'outlet_years': 4.0,
  'stats': 'min'},
 {'item_MRP': 93.8265,
  'item_outlet_sales': 834.2474,
  'item_visibility_mean_ratio': 0.9980230823,
  'item_weight': 8.785,
  'outlet_years': 9.0,
  'stats': '25%'},
 {'item_MRP': 143.0128,
  'item_outlet_sales': 1794.331,
  'item_visibility_mean_ratio': 1.0,
  'item_weigh

In [56]:
#train_df.drop('primary_key', axis=1, inplace=True)

train_corr = train_df.corr()

#reset index in order to stats column
train_corr = train_corr.reset_index()
#rename index to stats
train_corr.rename(columns={'index': 'stats'}, inplace=True)


train_corr

,stats,item_MRP,item_outlet_sales,item_weight,outlet_years,item_visibility_mean_ratio
0,item_MRP,1.000000,0.567574,0.026172,-0.005020,-0.003626
1,item_outlet_sales,0.567574,1.000000,0.013261,0.049135,-0.391115
2,item_weight,0.026172,0.013261,1.000000,0.012823,0.007643
3,outlet_years,-0.005020,0.049135,0.012823,1.000000,0.268271
4,item_visibility_mean_ratio,-0.003626,-0.391115,0.007643,0.268271,1.000000


In [58]:
train_corr_json = json.loads(train_corr.to_json(orient='records'))
train_corr_json 

[{'item_MRP': 1.0,
  'item_outlet_sales': 0.5675744467,
  'item_visibility_mean_ratio': -0.0036264272,
  'item_weight': 0.0261723221,
  'outlet_years': -0.0050199162,
  'stats': 'item_MRP'},
 {'item_MRP': 0.5675744467,
  'item_outlet_sales': 1.0,
  'item_visibility_mean_ratio': -0.3911151227,
  'item_weight': 0.0132612364,
  'outlet_years': 0.0491349704,
  'stats': 'item_outlet_sales'},
 {'item_MRP': 0.0261723221,
  'item_outlet_sales': 0.0132612364,
  'item_visibility_mean_ratio': 0.0076426537,
  'item_weight': 1.0,
  'outlet_years': 0.0128226467,
  'stats': 'item_weight'},
 {'item_MRP': -0.0050199162,
  'item_outlet_sales': 0.0491349704,
  'item_visibility_mean_ratio': 0.2682711371,
  'item_weight': 0.0128226467,
  'outlet_years': 1.0,
  'stats': 'outlet_years'},
 {'item_MRP': -0.0036264272,
  'item_outlet_sales': -0.3911151227,
  'item_visibility_mean_ratio': 1.0,
  'item_weight': 0.0076426537,
  'outlet_years': 0.2682711371,
  'stats': 'item_visibility_mean_ratio'}]